# The most important comments

### Psi4-MO to Cfour-MO in C1

* Row to row transformation of **C**:
  * Reordering is needed as l,m order is different.
  * Scaling is needed for some (l,m).
* On a given center:
  * Psi4  order: shell-first,          then m(Psi4-order)
  * Cfour order: m(Cfour-order)-first, then shell
  * m(Psi4-order) and m(Cfour-order) are different.
* To establish the Psi4-to-Cfour mapping vector `map_p2c`:
  * apply the Psi4-to-Cfour `m_map`
  * apply offsets to correct for m-first order in Cfour

### Mapping for one center, for one shell.

From `basis_mapping` documentation in `P4toC4_aux.py`

Goal: Establish Psi4-toCfour MO mapping/scaling operation, so that

`Cfour4_MO[map_p2c[i]] = Psi4_MO[i]/scale[i]`


for a single shell the mapping is (without any offsets):

m_map = where to put MO coefficient m in the Cfour vector
s_map = how to scale MO coefficient m

      Psi4-order      -> Cfour-order
p:    (0,1,2)         -> (2,0,1)   
d:    (0,1,2,3,4)     -> (0,2,4,3,1)
f:    (0,1,2,3,4,5,6) -> (2,0,1,6,4,3,5)

and the associated scaling-factors are:
sq12, sq24, sq40, sq60 = np.sqrt([12, 24, 40, 60])
s_map = np.array(
    [[1, 0, 0, 0, 0, 0, 0],
     [1, 1, 1, 0, 0, 0, 0],
     [sq12, 1, 1, 2, 1, 0, 0],
     [sq60, sq40, sq40, 2, 1, sq24, sq24]]
    )                                      

### Psi4-MO to Cfour-MO with Cs, C2, or Ci

* Both sets of MOs are in SO representation.
* For higher symmetries irrep-order will need a mapping.
* Any AO can contribute to only one SO[irrep], if at all.
* The first AO in an SO is relevant; the following are on symmetry-equivalent atoms.
* The mapping of the SOs is the arg-sorted Cfour-mapped first-AO list.
  * Create the first-AO list in Psi4 AO-order.
  * Create the first-AO list in Cfour AO-order (`p2c_map`).
  * Use `np.argsort` to find the Cfour-to-Psi4 SO mapping `so_c2p`.
  * Invert to find the Psi4-to-Cfour mapping `so_p2c`.
  * Use `so_p2c` to reorder the Psi4-MO vectors.
* At the same time the Psi4 MO coefficients must be scaled.
  * One factor is the AO-scaling also needed in C1.
  * The other factor is the SO normalization. Psi4 uses normalized SOs, Cfour does not.

### Running Cfour

* The best start orbitals will not help, if the guessed occupation (from core) is wrong.
* In these cases, the correct occupation must be added to the input as OCCUPATION statement. That should be done automatically in any Psi4-guess run.
* However, for higher symmetries, mind the irrep-mapping. 

### `OLDMOS` file

* To read start orbitals Cfour needs (beyond `ZMAT` and `GENBAS`):
  * `OLDMOS`: start orbitals *per se*.
  * `JFSGUESS`: an empty file serving as a flag.
  * The right `OCCUPATION` statement in `ZMAT`.

Format of the `OLDMOS` file:
1. Blocks of $\alpha$-MOs followed by blocks of $\beta$-MOs for UHF.
2. Blocks of MOs for each irrep.
3. Each spin-irrep-block of MOs is written as columns of four MOs at a time. 

Example: 
* 15 AOs combine to 11 $a^\prime$ and 4 $a^{\prime\prime}$ SOs. 
* The first, $\alpha$-$a^\prime$ block consists of 11 MOs in the $a^\prime$-SO representation. 
* The second, $\beta$-$a^{\prime\prime}$ block consists of 4 MOs in the $a^{\prime\prime}$-SO representation.
* For UHF, another two blocks ($a^\prime$ and $a^{\prime\prime}$ $\beta$) follow.
* The first, $\alpha$-$a^\prime$ block starts with 11 lines with 4 entries each for the first set of four MOs (4 column vectors). It is followed by another 11 lines with 4 entries for the second set of four MOs, and finally another 11 lines with entries for the set with the last three MOs.
* Each entry is written in the format: `30.20E`

